# Panic и recover

Система типов Go позволяет найти многие ошибки во время компиляции, но некоторые ошибки как выход за границу массива или разыменовывание пустого указателя требуют проверки во время выполнения программы. Когда runtime языка Go находит такого рода ошибки, оно **паникует**(panic).

Когда происходит паника, выполнение программы останавливается, выполняются все отложенные функции горутины, в которой произошла паника, и программа падает с выводом сообщения. Это сообщение содержит **значение паники**(panic value), которое обычно является сообщением об ошибке, и **трейс стека**(stack trace - стек вызовов функций) каждой активной во время паники горутины. Обычно этого сообщения бывает достаточно для определения причины паники.

Не всегда паника исходит от runtime'а. Встроенная функция **panic** позволяет вызвать панику напрямую. На вход она принимает значение любого типа. Панику обычно вызывают в случаях, когда происходит что-то совсем "невозможное".

Хотя механизм паник похож на механизм исключений из других языков программирования, ситуации, в которых вызывается паника, совершенно иные. Т.к. из-за паники программа завершает выполнение, паника в основном используются как посмертный лог, который подскажет разработчику причину падения. "Нормальные" ошибки, которые являются ожидаемыми программой, такие как неправильный ввод или настройка, ошибка при чтении данных из файла, невозможность сетевого подключения и т.д. не должны рассматриваться как исключения: программа должна уметь работать с такими ошибками.

Как уже говорилось, после паники выполняются все отложенные функции горутины, в которой возникла паника:

In [ ]:
import "fmt"

func f(n int) {
    fmt.Printf("f(%d)\n", n + 0/n) // Паника, когда n равно 0.
    defer fmt.Printf("defer %d\n", n)
    f(n - 1)
}

func main() {
    f(3)
}

Также в Go возможно **восстановиться** после паники. Если встроенная функция **recover** вызывается в отложенной(deferred) функции и функция, содержащая отложенную функцию, паникует, то recover останавливает процесс паники и возвращает значение паники. Паникующая функция не выполняется дальше места, где произошла паника, а завершается нормально, т.е. с возвратом значения. Если recover вызвать в любой другой момент, то она просто возвращает nil и ничего особенного не происходит. Пример:

In [6]:
import "fmt"

func foo() (value int, err error) {
    defer func() {
        if p := recover(); p != nil {
            err = fmt.Errorf("panic message: %v", p)
        }
    }()
    
    panic("STOP")
}

_, e := foo()
fmt.Sprint(e)

panic message: STOP


Восстановление от паники можно использовать для выполнения какого-либо последнего действия перед завершением программы. Считается плохой практикой без явной необходимости восстанавливаться от паники, т.к. после паники значение переменных пакета редко бывает определённым. Восстановление от паники внутри пакета может быть хорошим способом обработки сложной ошибки, но в общем случае не нужно пытаться восстановиться от ошибки в другом пакете или функции, которую не контролируете. Публичное API должно возвращать ошибки в случае неудачи. Не от всех паник можно восстановиться: нехватка памяти вызовет завершение программы в любом случае с выводом сообщения.